# Notes on Testing Notebooks

Following [a blog post by Alex Remedios](https://semaphoreci.com/blog/test-jupyter-notebooks-with-pytest-and-nbmake)

## Prereqs

```sh
make pip-notebooks
```

This installs the `require_extras.notebooks` dependencies into the virtual env 

## Testing the Notebook

Assuming all the notebooks are under `./notebooks` the following commands will work with Pytest

* Simple commmand:

    `pytest --nbmake notebooks`
* Distributed command (assuming `pytest-xdist` has been installed):

    `pytest --nbmake -n auto notebooks`
* Overwrite the notebooks (by default notebooks are untouched):

    `pytest --nbmake --overwrite notebooks`

### Example: Good Cell

In [1]:
print(f"""
{2**3**5=}
{(2**3)**5=}
{2**(3**5)=}
""")


2**3**5=14134776518227074636666380005943348126619871175004951664972849610340958208
(2**3)**5=32768
2**(3**5)=14134776518227074636666380005943348126619871175004951664972849610340958208



### Example: Bad Cell

By default, an erroring cell will fail the test. Here's an example run:

```sh
❯ pytest --nbmake  notebooks
=== test session starts ===
...
collected 2 items
                                       
notebooks/notebooks_test.ipynb F   [50%]                                                                                                           
notebooks/quadratic_factoring_game.ipynb . [100%]

=== FAILURES ===
/Users/zeph/github/algorand/graviton/notebooks/notebooks_test.ipynb
---------------------------------------------------------------------------
assert 2**3**5 == (2**3)**5, "yeah, I know this is wrong"
---------------------------------------------------------------------------
```

But if we set the path `metadata.execution.allows_errors` in the notebook's JSON to `true`, 
then even though the cell still behaves as expected, it doesn't halt `pytest`'s execution, and `pytest` passes as well.

IE: editing the notebook `*.ipynb` as JSON:

```json
# Set the path `metadata.execution.allows_errors = true`
{
    "cells": [ ... ],
    ...
    "metadata": {
      "execution": {
       "allow_errors": true
      },
    ...
}
```

Now when you run `pytest`, the notebook won't fail even though the cell errors:

```sh
❯ pytest --nbmake notebooks
=== test session starts ===
platform darwin -- Python 3.10.2, pytest-7.1.1, pluggy-1.0.0
rootdir: /Users/zeph/github/algorand/graviton
plugins: hypothesis-6.41.0, xdist-2.5.0, forked-1.4.0, nbmake-1.3.0
collected 2 items                                                                                                                                              

notebooks/notebooks_test.ipynb .           [ 50%]
notebooks/quadratic_factoring_game.ipynb . [100%]
```

In [2]:
assert 2**3**5 == (2**3)**5, "yeah, I know this is wrong"

AssertionError: yeah, I know this is wrong

In [3]:
print("I got here, even though the above failed")

I got here, even though the above failed
